In [ ]:
import random
import compyute as cp

In [ ]:
device = cp.cuda if cp.backend.gpu_available() else cp.cpu
device

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd

data = pd.read_csv('data/cifar10/train.csv')
data.head()

In [ ]:
tensor = cp.tensor(data.to_numpy())
train, val, test = cp.preprocessing.split_train_val_test(tensor, ratio_val=0.1, ratio_test=0.1)

In [ ]:
X_train, y_train = train[:, 1:], train[:, 0].to_int()
X_val, y_val = val[:, 1:], val[:, 0].to_int()
X_test, y_test = test[:, 1:], test[:, 0].to_int()

In [ ]:
X_train = cp.reshape(X_train, shape=(X_train.shape[0], 3 , 32, -1)).to_int()
X_val = cp.reshape(X_val, shape=(X_val.shape[0], 3, 32, -1)).to_int()
X_test = cp.reshape(X_test, shape=(X_test.shape[0], 3, 32, -1)).to_int()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(cp.movedim(X_train[0], 0, -1))

Scaling

In [ ]:
X_train = X_train.to_float() / 255.0
X_val = X_val.to_float() / 255.0
X_test = X_test.to_float() / 255.0

In [ ]:
from compyute import nn
from transformer.vit import VisionTransformer

model = VisionTransformer(
    in_channels=3,
    image_size=32,
    patch_size=4,
    embedding_dim=384,
    n_classes=10,
    ffwd_channels=4*384,
    n_heads=6,
    n_blocks=6,
    dropout=0.1
)

model.to_device(device)

In [ ]:
summary = cp.nn.utils.get_module_summary(model, input_shape=(3, 32, 32))
print(summary)

In [ ]:
from compyute.nn.trainer import Trainer
from compyute.nn.trainer.callbacks import History, ProgressBar

history = History()

trainer = Trainer(
    model=model,
    optimizer=nn.optimizers.Adam(lr=3e-4),
    loss=nn.CrossEntropy(),
    metric=nn.Accuracy(),
    callbacks=[
        history,
        ProgressBar(mode="step"),
    ]
)

In [ ]:
epochs = 5
batch_size = 128

trainer.train(X_train, y_train, epochs=epochs, batch_size=batch_size, val_data=(X_val, y_val))

In [ ]:
# ! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

def plot_history(t1, t2):
    trace1 = history[t1]
    trace2 = history[t2]
    plt.figure(figsize=(10, 3))
    plt.plot(cp.arange(start=1, stop=len(trace1) + 1), trace1, linewidth=1)
    plt.plot(cp.arange(start=1, stop=len(trace2) + 1), trace2, linewidth=1)

plot_history("loss", "accuracy_score")

In [ ]:
loss, accuracy = trainer.evaluate_model(X_test, y_test, batch_size)
print(f'loss {loss:.4f}')
print(f'accuracy {accuracy*100:.2f}')

In [ ]:
# ! pip install scikit-learn

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy
from compyute.nn.utils import batched

y_pred = batched(model, batch_size, model.device, False)(X_test)
probs = nn.functional.softmax(y_pred)

cm = confusion_matrix(
    y_true=y_test.to_numpy(),
    y_pred=cp.argmax(probs, dim=-1).to_numpy(),
    labels=cp.unique(y_test).to_numpy()
)

r = cp.arange(10).to_numpy()
plt.imshow(cm, cmap="Blues")
plt.xlabel("prediction")
plt.ylabel("target")
plt.xticks(ticks=r, labels=r)
plt.yticks(ticks=r, labels=r)
for (j, i), label in numpy.ndenumerate(cm):
    plt.text(i, j, str(int(label)), ha="center", va="center")